# Loading the datset from Kaggle and Unzipping the data


In [5]:
# Load dataset from kaggle and install necessary packages

!pip install kaggle


In [6]:
#importing necessary kaggle packages

import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

In [9]:
# kaggle api setip

# Setting up Kaggle API credentials
os.environ['KAGGLE_USERNAME'] = 'krishnaheroor'  #add your kaggle username
os.environ['KAGGLE_KEY'] = 'please add the api token for above username'


api = KaggleApi()
api.authenticate()

In [15]:
#mapping dataset from kaggle

dataset = 'datasets/face-expression-recognition-dataset'
download = 'datasets/face-expression-recognition-dataset.zip'

In [12]:
#loading the above dataset for project
api.dataset_download_files('jonathanoheix/face-expression-recognition-dataset', path='datasets', unzip=False)

Dataset URL: https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset


In [16]:
#unzipping the datsset
with zipfile.ZipFile(download, 'r') as zip_ref:
    zip_ref.extractall(dataset)

## Image pre-processing


In [17]:
#importing the necessary packages and libraries

import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [33]:
dataset

'datasets/face-expression-recognition-dataset'

In [39]:
#dataset preparatons

train_dir = '/content/datasets/face-expression-recognition-dataset/images/train'
val_dir = '/content/datasets/face-expression-recognition-dataset/images/validation'


('/content/datasets/face-expression-recognition-dataset/images/train',
 '/content/datasets/face-expression-recognition-dataset/images/validation')

In [40]:
#setting up image size/dimns
IMG_SIZE = 48

In [43]:
# Mapping string labels to numeric values
label_map = {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}

In [44]:
#load data

def load_data(data_dir):
    images = []
    labels = []
    for label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            if label in label_map:
                numeric_label = label_map[label]
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  #if in case gray scale images
                    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))  #resizing into 48x48 image size
                    img = img.astype('float32') / 255.0  #mapping between 0 to 1 as its a gray scale
                    images.append(img)
                    labels.append(numeric_label)
            else:
                print(f"Ignoring unknown label '{label}'")

    images = np.array(images).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
    labels = to_categorical(labels, num_classes=7)  # we have 7 classes
    return images, labels

In [45]:
# Load training and validation/test data
try:
    X_train, y_train = load_data(train_dir)
    X_val, y_val = load_data(val_dir)
    print("Data loaded successfully!")
except Exception as e:
    print(f"Error loading data: {e}")

Data loaded successfully!


In [ ]:
# Data Augmentation if required

# new_data_generate = ImageDataGenerator(
#     rotation_range=10,
#     zoom_range=0.1,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
    # vertical_flip=True,
    # horizontal_flip = True,
    # fill_mode='nearest'
# )

## Model Building

In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [47]:
#let initialize the model

model = Sequential()

In [48]:
#dense neural network architecture and stage creation


model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [49]:
# setthe model with necesasry back propogation techniques like optimizers, cost function etc..

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


#cost/loss function used is - categorical_crossentropy (As we converted to whole numerical part in above stage)
#optimizers - To achive gradient decent - Adam we used

## Model Training with Hyper parameters like epochs, learning rate etc

In [ ]:

# Train the model


# history_check = model.fit(
#     new_data_generate.flow(X_train, y_train, batch_size=64),
#     validation_data=(X_val, y_val),
#     epochs=30
# )   use this if you used data augmentation or else below code

In [50]:
history_check = model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/30
451/451 [==============================] - 99s 214ms/step - loss: 1.7392 - accuracy: 0.2904 - val_loss: 1.5620 - val_accuracy: 0.3889
Epoch 2/30
451/451 [==============================] - 89s 198ms/step - loss: 1.5286 - accuracy: 0.4097 - val_loss: 1.4457 - val_accuracy: 0.4533
Epoch 3/30
451/451 [==============================] - 90s 199ms/step - loss: 1.4201 - accuracy: 0.4523 - val_loss: 1.3269 - val_accuracy: 0.4956
Epoch 4/30
451/451 [==============================] - 89s 197ms/step - loss: 1.3505 - accuracy: 0.4866 - val_loss: 1.2786 - val_accuracy: 0.5221
Epoch 5/30
451/451 [==============================] - 95s 211ms/step - loss: 1.2952 - accuracy: 0.5086 - val_loss: 1.2355 - val_accuracy: 0.5355
Epoch 6/30
451/451 [==============================] - 95s 210ms/step - loss: 1.2408 - accuracy: 0.5300 - val_loss: 1.2182 - val_accuracy: 0.5470
Epoch 7/30
451/451 [==============================] - 94s 208ms/step - loss: 1.1990 - accuracy: 0.5483 - val_loss: 1.2267 - val_ac

## Save model

In [51]:
model.save('face_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Inference code for testing

In [60]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('face_model.h5')

# Define labels for emotions
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Function to classify emotion from an image file path or web cam
def classify_emotion_from_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load image from path: {image_path}")
        return None

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_resized = cv2.resize(img_gray, (48, 48))
    img_normalized = img_resized.astype('float32') / 255.0
    img_input = np.expand_dims(img_normalized, axis=-1)
    img_input = np.expand_dims(img_input, axis=0)

    # here we are predecting the results
    predictions = model.predict(img_input)
    predicted_label = np.argmax(predictions)

    # converting to array of numbers fro above predictions
    emotion = labels[predicted_label]

    return emotion

# only if you wnat to use webcam else comment the below code
# cap = cv2.VideoCapture(0)

# while True:
#     # Capture frame-by-frame from webcam
#     ret, frame = cap.read()

#     if not ret:
#         print("Failed to capture frame from webcam. Exiting...")
#         break

#     # Perform emotion classification on the webcam frame
#     predicted_emotion_webcam = classify_emotion(frame)

#     # Display the webcam frame with emotion prediction
#     cv2.putText(frame, f"Emotion (Webcam): {predicted_emotion_webcam}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#     cv2.imshow('Webcam Emotion Classification', frame)

#     # Check if 'q' key is pressed to exit webcam loop
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the webcam
# cap.release()

# Perform emotion classification on an image
image_path = '/content/datasets/face-expression-recognition-dataset/images/validation/angry/10121.jpg'  #give your image path for inference testing
predicted_emotion_image = classify_emotion_from_image(image_path)

# prediction here to check the lables of specific image
if predicted_emotion_image:
    print(f"Predicted Emotion (Image): {predicted_emotion_image}")


1/1 [==============================] - 0s 89ms/step
Predicted Emotion (Image): Angry
